In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('Divar.csv',low_memory=False)

In [3]:
df.shape

(1000000, 61)

In [4]:
df = df.loc[:,(df.isna().sum()<(df.shape[0]*.7)).values]
df.shape

(1000000, 38)

### cat2_slug , cat3_slug

In [5]:
# df.cat2_slug.value_counts()

In [6]:
df.cat3_slug.value_counts()

cat3_slug
apartment-sell                        303385
apartment-rent                        211880
plot-old                              133570
house-villa-sell                      121753
house-villa-rent                       64678
shop-rent                              45993
shop-sell                              21855
office-rent                            21418
suite-apartment                        16465
presell                                15781
villa                                  12899
industry-agriculture-business-sell     11851
industry-agriculture-business-rent      9155
office-sell                             5155
partnership                             3622
workspace                                539
Name: count, dtype: int64

In [7]:
mapping = {
    'apartment-sell':'apartment',
    'apartment-rent':'apartment',
    'plot-old':'plot-old',
    'house-villa-sell':'house-villa',
    'house-villa-rent':'house-villa',
    'shop-rent':'shop',
    'shop-sell':'shop',
    'office-rent':'office',
    'office-sell':'office',
    'suite-apartment':'suite-apartment',
    'presell':'presell',
    'villa':'villa',
    'industry-agriculture-business-sell':'industry-agriculture-business',
    'industry-agriculture-business-rent':'industry-agriculture-business',
    'partnership':'partnership',
    'workspace':'workspace'
}

In [8]:
df['cat3_slug'] = df.cat3_slug.map(mapping)
df.cat3_slug.value_counts()

cat3_slug
apartment                        515265
house-villa                      186431
plot-old                         133570
shop                              67848
office                            26573
industry-agriculture-business     21006
suite-apartment                   16465
presell                           15781
villa                             12899
partnership                        3622
workspace                           539
Name: count, dtype: int64

In [9]:
df.cat3_slug.value_counts()[df.cat3_slug.value_counts()<10000].index

Index(['partnership', 'workspace'], dtype='object', name='cat3_slug')

In [10]:
useless_values = df.cat3_slug.value_counts()[df.cat3_slug.value_counts()<1000].index
df = pd.concat([df , pd.get_dummies(df.cat3_slug)] , axis=1).drop(useless_values,axis=1)

In [11]:
df = df.drop(['cat2_slug','cat3_slug'],axis=1)

### city_slug , neighborhood_slug

In [12]:
city_categorization = pd.read_csv('iran_city_classification.csv')
city_categorization.columns

Index(['نام شهر', 'دسته‌بندی'], dtype='object')

In [13]:
city_categorization = city_categorization.set_index('نام شهر')

In [14]:
city_categorization.loc[['tehran','rasht']]

,دسته‌بندی
نام شهر,
tehran,کلان‌شهر
rasht,شهر کوچک


In [15]:
def calcCityCategory(row):
    if row['city_slug'] in city_categorization.index:
        return city_categorization.loc[row['city_slug']]['دسته‌بندی']
    return None

In [16]:
df['city_category'] = df.apply(calcCityCategory , axis=1)
df.city_category.value_counts()

city_category
شهر کوچک    498478
کلان‌شهر    464799
Name: count, dtype: int64

In [17]:
df.loc[:,'metropolis'] = df.city_category == 'کلان‌شهر'

In [18]:
df = df.drop(['city_slug','neighborhood_slug','city_category'] , axis=1)

In [19]:
df.metropolis.value_counts()

metropolis
False    535201
True     464799
Name: count, dtype: int64

### created_at_month

In [20]:
df.created_at_month = pd.to_datetime(df.created_at_month)
df.loc[:,'created_year'] = df.created_at_month.dt.year
df.loc[:,'created_month'] = df.created_at_month.dt.month
df.created_month.value_counts()

created_month
7     133319
8     132522
10    126606
6     125695
11    121819
9     121615
12    116998
5     108820
4       7215
1       2099
3       1912
2       1380
Name: count, dtype: int64

In [21]:
df.created_year.value_counts()

created_year
2024    996946
2023      1689
2025      1292
2022        65
2021         6
2020         2
Name: count, dtype: int64

In [22]:
df = df.drop('created_at_month',axis=1)

### user_type , description , title

In [23]:
df = df.drop(['description','title'] , axis=1)

### price cols

In [24]:
print(df.shape)
df = df[(~df.price_value.isna())|((~df.credit_value.isna())&(~df.rent_value.isna()))]
df.shape

(1000000, 44)


(919538, 44)

In [25]:
df.loc[:,'is_for_sell'] = ~df.price_value.isna()
df['is_for_sell'].value_counts()

is_for_sell
True     568346
False    351192
Name: count, dtype: int64

In [26]:
def calcPrice(row):
    if pd.isna(row['price_value']):
        return (row['credit_value'] / 1e6) * 30000 + row['rent_value']
    return row['price_value']
df.loc[:, 'price'] = df.apply(calcPrice, axis=1)
df[['price_value','credit_value','rent_value','price']].head()

,price_value,credit_value,rent_value,price
1,8.500000e+09,NaN,NaN,8.500000e+09
2,NaN,750000000.0,26000000.0,4.850000e+07
3,NaN,950000000.0,95000000.0,1.235000e+08
4,5.750000e+09,NaN,NaN,5.750000e+09
5,NaN,250000000.0,6000000.0,1.350000e+07


In [27]:
df = df.drop(['price_value','credit_value','rent_value'] , axis=1)

In [28]:
def calcMode(row):
    if pd.notna(row['price_mode']):
        return row['price_mode']
    if pd.notna(row['rent_mode']):
        return row['rent_mode']
    return row['credit_mode']
df.loc[:, 'mode'] = df.apply(calcMode , axis=1)
df.loc[:, 'fixed_price'] = df['mode'] == 'مقطوع'
df.fixed_price.value_counts()

fixed_price
True     858413
False     61125
Name: count, dtype: int64

In [29]:
df = df.drop(['price_mode','rent_mode','credit_mode','mode'] , axis=1)

In [30]:
df = df.drop(['rent_credit_transform','transformable_price','transformable_credit','transformable_rent'] , axis=1)

In [31]:
# df.fixed_price.isna().sum()

### floor , rooms_count , total_floors_count , unit_per_floor

In [32]:
def clean_floor(val):
    if pd.isna(val):
        return 0
    val = str(val).strip()
    
    # +30
    if val.endswith('+'):
        return int(val.replace('+', ''))  
    try:
        return int(float(val))  
    except ValueError:
        return np.nan

df.loc[:,'floor'] = df.floor.apply(clean_floor).astype('Int64')
df.floor.value_counts()

floor
0     415384
2     128170
1     119179
3     109008
4      71694
5      37417
6      13717
7       5674
-1      4427
8       3264
10      2284
9       2167
11      1461
12      1224
13       826
14       780
30       735
15       447
16       310
17       268
20       212
18       186
19       151
21       115
25       102
22        87
23        76
24        61
27        35
28        32
26        25
29        20
Name: count, dtype: int64

In [33]:
mapping = {
    'بدون اتاق' : 0,
    'یک' : 1,
    'دو' : 2,
    'سه' : 3,
    'چهار' : 4,
    'پنج یا بیشتر' : 5
}
df.loc[:,'rooms_count'] = df.rooms_count.map(mapping)

In [34]:
df.rooms_count.value_counts()

rooms_count
2.0    391251
1.0    179542
3.0    134634
0.0     69430
4.0     20335
5.0     12588
Name: count, dtype: int64

In [35]:
df.loc[:,'rooms_count'] = df.rooms_count.fillna(0)

C:\Users\ROG\AppData\Local\Temp\ipykernel_4480\1454943812.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.loc[:,'rooms_count'] = df.rooms_count.fillna(0)


In [36]:
df.rooms_count.isna().sum()

0

In [37]:
df = df.drop(['total_floors_count','unit_per_floor'] , axis=1)

### has...

In [38]:
df.has_restroom.value_counts()

has_restroom
squat_seat    219062
squat         179259
seat            7409
unselect         628
Name: count, dtype: int64

In [39]:
df.has_restroom.isna().sum()

513180

In [40]:
df.loc[:,'has_balcony'] = df.has_balcony.isin(['true','True']).astype(bool)

In [41]:
df.loc[:, 'has_elevator'] = (df.has_elevator.astype(bool).fillna(False))

In [42]:
df.loc[:, 'has_warehouse'] = (df.has_warehouse.astype(bool).fillna(False))

In [43]:
df.loc[:, 'has_parking'] =(df.has_parking.astype(bool).fillna(False))

In [44]:
df.loc[:,'has_warm_water_provider'] = df.has_warm_water_provider.isin(['package','water_heater','powerhouse']).astype(bool)

In [45]:
df.loc[:,'has_heating_system'] = df.has_heating_system.isin(['shoofaj','heater','duct_split','floor_heating','split','fan_coil','fireplace']).astype(bool)

In [46]:
df.loc[:,'has_cooling_system'] = df.has_cooling_system.isin(['water_cooler','air_conditioner','split','duct_split','fan_coil']).astype(bool)

In [47]:
df.loc[:,'has_restroom'] = df.has_restroom.isin(['squat_seat','squat','seat']).astype(bool)

In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 919538 entries, 1 to 999999
Data columns (total 35 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   Unnamed: 0                     919538 non-null  int64  
 1   building_size                  918254 non-null  float64
 2   floor                          919538 non-null  object 
 3   rooms_count                    919538 non-null  object 
 4   has_balcony                    919538 non-null  object 
 5   has_elevator                   919538 non-null  object 
 6   has_warehouse                  919538 non-null  object 
 7   has_parking                    919538 non-null  object 
 8   construction_year              807628 non-null  object 
 9   is_rebuilt                     529237 non-null  object 
 10  has_warm_water_provider        919538 non-null  object 
 11  has_heating_system             919538 non-null  object 
 12  has_cooling_system             9195

### construction_year , is_rebuilt

In [49]:
# df.is_rebuilt.value_counts()

In [50]:
# df.is_rebuilt.isna().sum()

In [51]:
def convert_persian_year(year_str):
    if pd.isna(year_str):
        return np.nan
    
    year_str = str(year_str).strip()
    
    if 'قبل از ۱۳۷۰' in year_str:
        return 1369  

    persian_to_english = str.maketrans('۰۱۲۳۴۵۶۷۸۹', '0123456789')
    year_english = year_str.translate(persian_to_english)
    year_digits = ''.join(filter(str.isdigit, year_english))
    
    if year_digits:
        return int(year_digits)
    else:
        return np.nan

df.loc[:,'construction_year'] = df['construction_year'].apply(convert_persian_year)

In [52]:
mode_value = df.construction_year.mode()[0]
df['construction_year'] = (pd.to_numeric(df['construction_year'], errors='coerce').fillna(mode_value).astype('Int64'))

In [53]:
df['is_rebuilt'] = df['is_rebuilt'].astype('boolean').fillna(False)

### building_direction , floor_material

In [54]:
# df.floor_material.value_counts()

In [55]:
# df.floor_material.isna().sum()

In [56]:
df = df.drop(['building_direction','floor_material'],axis=1)

### location_latitude , location_longitude , location_radius 

In [57]:
df.location_radius.isna().sum()

596447

In [58]:
df = df.drop('location_radius' , axis=1)

In [59]:
print(df.shape)
df = df.dropna(subset=['location_latitude', 'location_longitude'])
df.shape

(919538, 32)


(606910, 32)

In [60]:
df.shape

(606910, 32)

In [61]:
import geopandas as gpd
from shapely.geometry import Point


gdf = gpd.GeoDataFrame(
    df, 
    geometry=gpd.points_from_xy(df['location_longitude'], df['location_latitude']),
    crs="EPSG:4326"   
)

world = gpd.read_file("ne_110m_admin_0_countries/ne_110m_admin_0_countries.shp")


iran = world[world['NAME'] == "Iran"]

gdf = gdf[gdf.within(iran.iloc[0].geometry)]

df = gdf.drop(columns='geometry')

In [62]:
df.shape

(602030, 32)

In [63]:
from pyproj import Transformer
transformer = Transformer.from_crs('epsg:4326','epsg:32639',always_xy=True)

df['utm_x'], df['utm_y'] = transformer.transform(
    df["location_longitude"].values, 
    df["location_latitude"].values
)

df[["location_latitude", "location_longitude", "utm_x", "utm_y"]].head()

,location_latitude,location_longitude,utm_x,utm_y
2,35.703865,51.373459,533784.424602,3.951168e+06
7,35.729832,51.505466,545711.558185,3.954101e+06
8,35.712364,50.794781,481437.130969,3.952066e+06
10,35.778664,51.757549,568467.028494,3.959664e+06
11,35.733952,51.380608,534418.187237,3.954507e+06


In [64]:
df = df.drop(['location_latitude','location_longitude'] , axis=1)

### land_size , building_size

In [65]:
df = df.drop('Unnamed: 0',axis=1)

In [66]:
# df.info()

In [67]:
cols = np.abs(df.drop('price',axis=1).corr()['building_size']).sort_values(ascending=False)[1:16].index

In [68]:
X_train = df[~df.building_size.isna()][cols]
y_train = df[~df.building_size.isna()]['building_size']

In [69]:
X_train.isna().sum()

industry-agriculture-business    0
apartment                        0
has_heating_system               0
has_restroom                     0
has_warm_water_provider          0
has_cooling_system               0
has_balcony                      0
utm_y                            0
floor                            0
metropolis                       0
is_rebuilt                       0
rooms_count                      0
has_elevator                     0
construction_year                0
has_parking                      0
dtype: int64

In [70]:
from sklearn.linear_model import LinearRegression

linear_regression = LinearRegression()
linear_regression.fit(X_train,y_train)

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [71]:
X_missing = df[df.building_size.isna()][cols]
y_pred_missing = linear_regression.predict(X_missing).astype(float)

df.loc[df.building_size.isna(), 'building_size'] = y_pred_missing

In [72]:
df[df.building_size<0]

,building_size,floor,rooms_count,has_balcony,has_elevator,has_warehouse,has_parking,construction_year,is_rebuilt,has_warm_water_provider,...,suite-apartment,villa,metropolis,created_year,created_month,is_for_sell,price,fixed_price,utm_x,utm_y
110286,-175.030532,0,0.0,False,True,True,True,1403,False,False,...,False,False,True,2024,5,True,5.500000e+09,True,83033.419851,4.224197e+06
159292,-180.344848,0,0.0,False,True,True,True,1403,False,False,...,False,False,True,2024,8,True,3.222500e+09,True,85057.777996,4.225587e+06
163070,-173.657421,0,0.0,False,True,True,True,1403,False,False,...,False,False,True,2024,7,True,7.770000e+09,True,82848.797694,4.223837e+06
175855,-181.500639,0,0.0,False,True,True,True,1403,False,False,...,False,False,True,2024,7,True,4.765000e+09,True,86166.831063,4.225889e+06
208206,-76.653513,0,0.0,False,True,True,True,1403,False,False,...,False,False,True,2024,8,True,6.120000e+09,True,91772.843308,4.198462e+06
221321,-147.917362,0,0.0,False,True,True,True,1403,False,False,...,False,False,True,2024,6,True,7.600000e+09,True,98104.355992,4.217104e+06
229215,-164.938727,0,0.0,False,True,True,True,1403,False,False,...,False,False,True,2024,7,True,8.200000e+09,True,93059.044795,4.221557e+06
231124,-310.244286,0,0.0,False,True,True,True,1403,False,False,...,False,False,True,2024,11,True,9.700000e+09,True,532371.374048,3.948387e+06
290965,-147.948898,0,0.0,False,True,True,True,1403,False,False,...,False,False,True,2024,7,True,7.140000e+09,True,98669.469323,4.217112e+06
292507,-175.526991,0,0.0,False,True,True,True,1403,False,False,...,False,False,True,2024,5,True,6.960000e+09,True,87198.220490,4.224326e+06


In [73]:
df = df[df.building_size>0]

### saving 

In [74]:
df.isna().sum()

building_size                    0
floor                            0
rooms_count                      0
has_balcony                      0
has_elevator                     0
has_warehouse                    0
has_parking                      0
construction_year                0
is_rebuilt                       0
has_warm_water_provider          0
has_heating_system               0
has_cooling_system               0
has_restroom                     0
apartment                        0
house-villa                      0
industry-agriculture-business    0
office                           0
partnership                      0
plot-old                         0
presell                          0
shop                             0
suite-apartment                  0
villa                            0
metropolis                       0
created_year                     0
created_month                    0
is_for_sell                      0
price                            0
fixed_price         

In [75]:
# df.info()

In [76]:
df.describe()

,building_size,construction_year,created_year,created_month,price,utm_x,utm_y
count,6.019960e+05,601996.0,601996.000000,601996.000000,6.019960e+05,6.019960e+05,6.019960e+05
mean,2.212085e+03,1394.709837,2024.001091,8.579300,2.299476e+10,5.588312e+05,3.881567e+06
std,9.612007e+04,8.444107,0.046440,2.234786,1.751244e+12,2.865385e+05,2.509258e+05
min,1.000000e+00,1369.0,2022.000000,1.000000,0.000000e+00,-9.094646e+04,2.820041e+06
25%,7.300000e+01,1390.0,2024.000000,7.000000,1.900000e+07,4.781838e+05,3.833649e+06
50%,1.000000e+02,1397.0,2024.000000,9.000000,1.200000e+09,5.317325e+05,3.953346e+06
75%,1.500000e+02,1402.0,2024.000000,10.000000,3.604475e+09,5.705007e+05,4.035999e+06
max,1.000000e+07,1403.0,2025.000000,12.000000,1.030000e+15,1.661267e+06,4.385966e+06


In [77]:
df[df.price==0]

,building_size,floor,rooms_count,has_balcony,has_elevator,has_warehouse,has_parking,construction_year,is_rebuilt,has_warm_water_provider,...,suite-apartment,villa,metropolis,created_year,created_month,is_for_sell,price,fixed_price,utm_x,utm_y
20,210.0,0,4.0,True,True,True,True,1403,False,False,...,False,False,False,2024,11,True,0.0,False,3.574301e+05,3.702682e+06
877,144.0,0,1.0,False,True,True,False,1369,False,False,...,False,False,True,2024,12,True,0.0,False,1.384972e+05,3.803624e+06
1243,75.0,1,2.0,True,False,True,False,1395,True,True,...,False,False,False,2024,7,True,0.0,False,3.506455e+05,4.121707e+06
1297,200.0,0,3.0,True,True,False,True,1374,False,True,...,False,False,False,2024,10,True,0.0,False,8.521016e+05,3.153935e+06
2508,38.0,0,0.0,False,False,False,False,1375,False,False,...,False,False,True,2024,7,True,0.0,False,1.405776e+05,3.807576e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995382,200.0,0,2.0,True,True,True,True,1402,False,False,...,False,False,True,2024,7,True,0.0,False,2.713091e+05,3.471076e+06
995875,400.0,0,3.0,True,True,True,True,1400,False,True,...,False,False,False,2024,12,True,0.0,False,5.098659e+05,4.060758e+06
996500,75.0,1,2.0,True,False,False,True,1385,True,True,...,False,False,True,2024,7,True,0.0,False,1.264437e+06,4.059464e+06
998951,90.0,4,2.0,True,True,True,True,1397,False,True,...,False,False,True,2024,9,True,0.0,False,4.830012e+05,3.822757e+06


In [78]:
df.to_csv('Divar_preproccessed.csv')